In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install snscrape
!pip install wordcloud
!pip3 install textblob
!pip3 install imblearn

import snscrape.modules.twitter as sntwitter
import itertools
import tweepy
import csv
import datetime
import time
import tweepy

from textblob import TextBlob
from wordcloud import WordCloud
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict
from datetime import date
import termcolor
import sys
from termcolor import colored, cprint
plt.style.use('fivethirtyeight')

import re # for regular expressions
import string
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [ ]:
from nltk.stem.snowball import SnowballStemmer 
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
import string
import re  # for regular expressions

In [ ]:
#Install package for Emotion Analysis
!pip install text2emotion
#Import the modules
import text2emotion as te

#Install package for NB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.svm import SVC
# import method for split train/test data set
from sklearn.model_selection import train_test_split
# import method to calculate metrics
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

# Packages for LSI
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

#imblearn packages for smote
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from collections import Counter

# Import packages
!pip install mglearn
import warnings
warnings.filterwarnings('ignore')
import mglearn
import os
import glob
import pickle


# DATA COLLECTION 

We extract a total of 30000 tweets.

In [ ]:
def collect_data(dates):
    date_initial = [['2019-11-30', '2020-05-30',5000]]
    for i in range(len(date_initial)):
        print(date_initial[i][0],date_initial[i][1],date_initial[i][2])
        df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('"stress + since:"'+ date_initial[i][0] +'"until:"'+ date_initial[i][1] + '"lang:en"').get_items(), date_initial[i][2]))
    print(dates)
    print(len(dates))
    for i in range(len(dates)):
        print(dates[i][0],dates[i][1],dates[i][2])
        df=df.append(pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('"stress + since:"'+ dates[i][0] +'"until:"'+ dates[i][1] + '"lang:en"').get_items(), dates[i][2])))
        
    print(df)    
    df.to_csv("/content/drive/MyDrive/Colab Notebooks/BIA/tweets104.csv", index=False)
    return df


In [ ]:
date_list = [['2020-05-30','2020-11-30',10000],['2020-11-30','2021-05-30',10000],['2021-05-30','2021-10-15',5000]]
collect_data(date_list)

In [ ]:
#Drop unwanted columns
df = df.drop(['tcooutlinksss','outlinksss', 'outlinks', 'tcooutlinks'],axis=1)
df

In [ ]:
read_file = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/BIA/tweets103.csv")
read_file

# TEXT PREPROCESSING

In [ ]:

from bs4 import BeautifulSoup
nltk.download('wordnet')
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#remove duplicated values
read_file.drop_duplicates(inplace=True) 


def load_dict_smileys():

    return {
        ":‑)":"",
        ":-]":"",#"smiley",
        ":-3":"",#"smiley",
        ":->":"",#"smiley",
        "8-)":"",#"smiley",
        ":-}":"",#"smiley",
        ":)":"",#"smiley",
        ":]":"",#"smiley",
        ":3":"",#"smiley",
        ":>":"",#"smiley",
        "8)":"",#"smiley",
        ":}":"",#"smiley",
        ":o)":"",#"smiley",
        }

def load_dict_contractions():

    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        }

# the above said preprocessing functions
##
def remove_url(text):
  return re.sub(r'https?://\S+|www\.\S+', repl=r'', string=text)


def remove_numbers(text):
  return ''.join([c for c in text if not c.isdigit()])


def remove_punct(text):
  return ''.join([c for c in text if c not in string.punctuation])


def remove_stopword(text):
  return ' '.join([word for word in text.split() if word not in stopwords.words('english')])


def correct_spelling(text):
  return str(TextBlob(str(text)).correct())


def do_lemmatize(text):
  return ' '.join([lemmatizer.lemmatize(str(word), "v") for word in text.split()])


def html (text): 
    #Escaping HTML characters
    tweet = BeautifulSoup(text, "lxml").get_text()
    return tweet


def remove_mention (text): 
    # Removing the @
    return re.sub(r"(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", '', string=text)


In [ ]:

EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}


def emoticon_to_word(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
        text = text.replace('_', ' ')
    return text


In [ ]:
# emojis to words
##
!pip install demoji

import demoji
demoji.download_codes()

In [ ]:
# convert emojis to words
##
def emoji_to_word(text):
  emojis = demoji.findall(text)
  for emoji in emojis.keys():
    text = text.replace(emoji, emojis.get(emoji))
  return text


In [ ]:
# lowercase
read_file['clean_text'] = read_file['content'].apply(str.lower)

#remove html
read_file['clean_text'] = read_file['clean_text'].apply(html)

#remove mentions
read_file['clean_text'] = read_file['clean_text'].apply(remove_mention)

# remove urls
##
read_file['clean_text'] = read_file['clean_text'].apply(remove_url)

# remove numbers
##
read_file['clean_text'] = read_file['clean_text'].apply(remove_numbers)

# remove punctuations
##
read_file['clean_text'] = read_file['clean_text'].apply(remove_punct)

# remove stopwords
##
read_file['clean_text'] = read_file['clean_text'].apply(remove_stopword)

# convert emoticons to words
##
read_file['clean_text'] = read_file['clean_text'].apply(emoticon_to_word)

# convert emojis to words
##
read_file['clean_text'] = read_file['clean_text'].apply(emoji_to_word)



In [ ]:
# lemmatization
##
read_file['text_preprocessed'] = read_file['clean_text'].apply(do_lemmatize)


# remove short words
read_file['text_preprocessed'] = read_file['text_preprocessed'].apply(lambda x: ' '.join([w for w in str(x).split() if len(w)>3]))


read_file['tokenized_tweet'] = read_file.apply(lambda row: nltk.word_tokenize(row['text_preprocessed']), axis=1)


#read_file['text_preprocessed'] = tokenized_tweet

read_file.head()

In [ ]:
pos = read_file['text_preprocessed']
print(pos)

In [ ]:
#Stopwords
stop = stopwords.words('english')
print(stop)

In [ ]:
# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(pos):
    tags = pos_tag(pos)
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

read_file['POS tagged'] = pos.apply(token_stop_pos)
read_file

In [ ]:
read_file = read_file.drop(['POS tagged'],axis=1)
read_file

In [ ]:
# Medical Words
nlp = spacy.load('en_core_sci_md')
def disease_words(disease):
    disease1 = nlp(disease)
    disease2 = [w.text for w in disease1.ents]
    return disease2


In [ ]:
read_file['Medical Words'] = read_file['Lemma'].apply(disease_words)

In [ ]:
read_file.shape

# EXPLORATORY DATA ANALYSIS

Let's do a few visualizations and NLP operations 

In [ ]:
# getting the date column ready for datetime operations
read_file['date']= pd.to_datetime(read_file['date'])

In [ ]:
#Data Visualisation
content_df = read_file['content']
print(len(content_df))
content_df

In [ ]:
tweet_len = []
for contentRow in content_df:
    lengthofTweet = len(contentRow.split())
    tweet_len.append(lengthofTweet)

In [ ]:
read_file.insert(2,"Word Length of Tweet", tweet_len)

In [ ]:
print('Average number of words used per tweet:', round(read_file['Word Length of Tweet'].mean(),2))
print('Spread of number of words used per tweet:',round(read_file['Word Length of Tweet'].std(),2))

In [ ]:
character_tweet_len = []
for contentRow in content_df:
    lengthofTweet = len(contentRow.split())
    character_tweet_len.append(lengthofTweet)

In [ ]:
read_file.insert(2,"Character Length of Tweet", character_tweet_len)

In [ ]:
print('Average number of words used per tweet:', round(read_file['Character Length of Tweet'].mean(),2))
print('Spread of number of words used per tweet:',round(read_file['Character Length of Tweet'].std(),2))
print('Minimum number of number of words used per tweet:',read_file['Character Length of Tweet'].min())
print('Maximum number of words used per tweet:',read_file['Character Length of Tweet'].max())

In [ ]:
import matplotlib.pyplot as plt
plt.hist(read_file['Word Length of Tweet'], bins=50)
plt.ylabel('Count')
plt.xlabel('Word Length of Tweet');
plt.show()

In [ ]:
source_L = read_file['sourceLabel'].value_counts(ascending=True)
source_num = source_L.tail()

In [ ]:
source_num.plot.pie(subplots=True, figsize=(15, 10),autopct='%1.0f%%',title="Source of Tweets")
plt.legend(loc="lower right", fontsize=10,prop={'size': 12})
print("Other sources = 1885")

In [ ]:
df_1try = read_file.groupby('date')[["Character Length of Tweet","Word Length of Tweet"]].mean()


In [ ]:
read_file.groupby('sourceLabel')[["Word Length of Tweet"]].mean()\
.plot(kind='line', figsize=(20,10))\
.legend(loc='center left', bbox_to_anchor=(1, 0.5));  # set legend

## The most used sources for tweeting about stress

In [ ]:
def plot_frequency_charts(df, user, content):
    read_file = pd.DataFrame()
    read_file[user] = df[user]
    
    f, ax = plt.subplots(1,1, figsize=(16,4))
    total = float(len(df))
    g = sns.countplot(df[user], order = df[user].value_counts().index[:20])
    g.set_title("Number and percentage of {}".format(content))

    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(100*height/total),
                ha="center") 

    plt.title('Frequency of {} tweeting about Corona'.format(user))
    plt.ylabel('Frequency', fontsize=12)
    plt.xlabel(content, fontsize=12)
    plt.xticks(rotation=90)
    plt.show()


In [ ]:
plot_frequency_charts(read_file, 'sourceLabel','Source')

In [ ]:
user = read_file['user']
user

In [ ]:
# A function to count tweets based on regular expressions
def count_tweets(reg_expression, tweet):
    tweets_list = re.findall(reg_expression, tweet)
    return len(tweets_list)

In [ ]:
# Creating a dictionary to hold these counts
content_count = {
    'words' : read_file['content'].apply(lambda x: count_tweets(r'\w+', x)),
    'mentions' : read_file['content'].apply(lambda x: count_tweets(r'@\w+', x)),
    'hashtags' : read_file['content'].apply(lambda x: count_tweets(r'#\w+', x)),
    'urls' : read_file['content'].apply(lambda x: count_tweets(r'http.?://[^\s]+[\s]?', x)),   
}

In [ ]:
read_file = pd.concat([read_file, pd.DataFrame(content_count)], axis=1)
read_file

# SENTIMENT ANALYSIS

In [ ]:
#Subjectivity
def getsubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [ ]:
#Polarity to know how positive or negative the text is
def getpolarity(text):
     return TextBlob(text).sentiment.polarity

In [ ]:
#Create 2 new columns 

#read_file.insert(7,"Subjectivity", clean_file.apply(getsubjectivity))
#read_file.insert(8,"Polarity", clean_file.apply(getpolarity))

read_file['Subjectivity'] = read_file['text_preprocessed'].apply(getsubjectivity)
read_file['Polarity'] = read_file['text_preprocessed'].apply(getpolarity)


In [ ]:
#Compue negative, neutral and positive analysis
def analysis(points):
    if points < 0:
        return 'Negative'
    elif points == 0:
        return 'Neutral'
    else:
        return 'Positive'
read_file['SentimentAnalysis'] = read_file['Polarity'].apply(analysis)
read_file

In [ ]:
#All the positive Tweets
i = 1
sorted_data = read_file.sort_values(by=['Polarity'])
positive = []
for j in range(0, sorted_data.shape[0]):
    if (sorted_data['SentimentAnalysis'][j] == 'Positive'):
        positive += str(i)
        print(str(i) + ')' + sorted_data['content'][j])
        print()
        i+=1

In [ ]:
#All the negative Tweets
i = 1
sorted_data = read_file.sort_values(by=['Polarity'], ascending='False')
for j in range(0, sorted_data.shape[0]):
    if (sorted_data['SentimentAnalysis'][j] == 'Negative'):
        
        print(str(i) + ')' + sorted_data['content'][j])
        print()
        i+=1

In [ ]:
read_file['SentimentAnalysis'].value_counts()

In [ ]:
# Plot Subjectivity and Polarity

plt.figure(figsize=(8,6))
for i in range(0, read_file.shape[0]):
    plt.scatter(read_file['Polarity'][i], read_file['Subjectivity'][i], color='violet')
    
plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
#Positive tweets percentage
positive_tweets = read_file[read_file.SentimentAnalysis == 'Positive']
positive_tweets = positive_tweets['content']

round((positive_tweets.shape[0] / read_file.shape[0])* 100, 1)

In [ ]:
#Negative tweets percentage
negative_tweets = read_file[read_file.SentimentAnalysis == 'Negative']
negative_tweets = negative_tweets['content']

round((negative_tweets.shape[0] / read_file.shape[0])* 100, 1)

In [ ]:
#Visualize the count

read_file['SentimentAnalysis'].value_counts()

plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
read_file['SentimentAnalysis'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
#Word Cloud
words = ' '.join([tweets for tweets in read_file['text_preprocessed']])
Cloud = WordCloud(background_color = 'white', max_words=500, colormap='Set2', width = 3000, height = 2000, random_state = 99, max_font_size = 500).generate(words)

plt.imshow(Cloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Generate a word cloud image
stopwords = set(STOPWORDS)
mask = np.array(Image.open("/content/drive/MyDrive/Colab Notebooks/BIA/m.png"))
wordcloud = WordCloud(stopwords=stopwords,width = 3000, height = 2000, background_color="white", max_words=1000, mask=mask).generate(' '.join(clean_file['Lemma']))
# create twitter image
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
# store to file
plt.savefig("twitter.png", format="png")
plt.show()

# Binary classification using SVM & Logistic Regression algorithms to classify if a tweet is made from a bot or not

In [ ]:
svm_read_file = read_file
svm_read_file = svm_read_file .drop(["url","date","Character Length of Tweet","Word Length of Tweet","renderedContent", "id","user", "replyCount", "retweetCount"], axis=1)
svm_read_file = svm_read_file.drop(["likeCount","conversationId","lang","coordinates","place","text_preprocessed","clean_text"], axis=1)
svm_read_file = svm_read_file.drop(["words","mentions","hashtags","urls","Subjectivity", "Polarity", "SentimentAnalysis","tp_a"],axis=1)

In [ ]:
svm_read_file["sourceLabel"]

In [ ]:
## If binary source not in dataframe
l = []
for source in svm_read_file.sourceLabel:
    if source == "TweetDeck" or source == "Twitter for iPad" or source == "Twitter Web App" or source == "Twitter for Android" or source == "Twitter for iPhone":
        l.append(0)
    else:
        l.append(1)
svm_read_file.insert(2, "binarySource", l, True )


In [ ]:
X = svm_read_file["content"]
y = svm_read_file["binarySource"]
from sklearn.feature_extraction.text import TfidfVectorizer
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
tfidf_vect = TfidfVectorizer(stop_words=None, min_df=1) 
X_train= tfidf_vect.fit_transform(X_train)
X_test = tfidf_vect.transform(X_test)

In [ ]:
sm = SMOTE(random_state=3)
x_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

print("Before SMOTE :" , Counter(y_train))
print("After SMOTE :" , Counter(y_train_smote))

In [ ]:
clf = svm.LinearSVC(C=1.0).fit(x_train_smote, y_train_smote)
y_pred=clf.predict(X_test)
labels=sorted(test_y.unique())
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test.values, y_pred))
precision, recall, fscore, support=precision_recall_fscore_support(y_test.values, y_pred, labels=labels)

In [ ]:
decision_scores = clf.decision_function(X_test)
fpr, tpr, thresholds = roc_curve(y_test.values, decision_scores, pos_label=1)
auc_score = round(auc(fpr, tpr)*100,2)
prc_score = round(average_precision_score(y_test.values, decision_scores) *100,2)

print("AUC: ", auc_score,"%", "PRC: ", prc_score,"%")
plt.figure();
plt.plot(fpr, tpr, color='darkorange', lw=2);
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')    
plt.xlim([0.0, 1.0]);
plt.ylim([0.0, 1.05]);
plt.xlabel('False Positive Rate');
plt.ylabel('True Positive Rate');
plt.title('SVM-AUC');
plt.show();

precision, recall, thresholds = precision_recall_curve(y_test.values, decision_scores, pos_label=1)

plt.figure();
plt.plot(recall, precision, color='darkorange', lw=2);
plt.xlim([0.0, 1.0]);
plt.ylim([0.0, 1.05]);
plt.xlabel('Recall');
plt.ylabel('Precision');
plt.title('SVM-PRC');
plt.show();

In [ ]:
from sklearn.linear_model import LogisticRegression
clf2 = LogisticRegression().fit(x_train_smote, y_train_smote)
y_pred=clf2.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test.values, y_pred))


In [ ]:
decision_scores = clf2.decision_function(X_test)
fpr, tpr, thresholds = roc_curve(y_test.values, decision_scores, pos_label=1)
auc_score = round(auc(fpr, tpr)*100,2)
prc_score = round(average_precision_score(y_test.values, decision_scores) *100,2)

print("AUC: ", auc_score,"%", "PRC: ", prc_score,"%")
plt.figure();
plt.plot(fpr, tpr, color='darkorange', lw=2);
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')    
plt.xlim([0.0, 1.0]);
plt.ylim([0.0, 1.05]);
plt.xlabel('False Positive Rate');
plt.ylabel('True Positive Rate');
plt.title('SVM-AUC');
plt.show();

precision, recall, thresholds = precision_recall_curve(y_test.values, decision_scores, pos_label=1)

plt.figure();
plt.plot(recall, precision, color='darkorange', lw=2);
plt.xlim([0.0, 1.0]);
plt.ylim([0.0, 1.05]);
plt.xlabel('Recall');
plt.ylabel('Precision');
plt.title('SVM-PRC');
plt.show();

# TOPIC MODELLING - LDA

In [ ]:
print(read_file.columns)
print(read_file.shape)

In [ ]:
read_file['text_preprocessed'].head()

In [ ]:
stop = stopwords.words('english')
print(stop)

In [ ]:
def tokenize(text):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text) 
    filtered_tokens = [w for w in word_tokens if not w in stop_words if len(w) > 2]
    return filtered_tokens

vectorizer = TfidfVectorizer(tokenizer=tokenize, use_idf=True,smooth_idf=True)

svd_model = TruncatedSVD(n_components=10, algorithm='randomized',n_iter=10)

svd_transformer = Pipeline([('tfidf', vectorizer), ('svd', svd_model)])

svd_matrix = svd_transformer.fit_transform(read_file.text_preprocessed)

tfidf = svd_transformer.steps[0][-1]
voc = tfidf.get_feature_names()

features_names = np.array(voc)

sorting = np.argsort(svd_model.components_, axis=1)[:, ::-1]

mglearn.tools.print_topics(topics=range(10), feature_names=features_names,
                           sorting=sorting, topics_per_chunk=5, n_words=50)

# EMOTION ANALYSIS

In [ ]:
read_file['emotion'] = read_file.apply(lambda row: te.get_emotion(row['text_preprocessed']), axis=1)
read_file

In [ ]:
read_file['emotion']

In [ ]:
read_file['Happy']=[dict(i)['Happy'] for i in read_file['emotion']]
read_file['Angry']=[dict(i)['Angry'] for i in read_file['emotion']]
read_file['Surprise']=[dict(i)['Surprise'] for i in read_file['emotion']]
read_file['Sad']=[dict(i)['Sad'] for i in read_file['emotion']]
read_file['Fear']=[dict(i)['Fear'] for i in read_file['emotion']]
dominant=[]
for i in read_file['emotion']:
    p=dict(i)
    Keymax = max(p, key=p.get)
    dominant.append(Keymax)
read_file['dominant emotion'] = dominant

In [ ]:
read_file

In [ ]:
#Visualize the count

read_file['dominant emotion'].value_counts()

plt.title('Emotion Analysis')
plt.xlabel('Emotion')
plt.ylabel('Counts')
read_file['dominant emotion'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
dd=read_file[read_file['dominant emotion']=='Happy']
text=" ".join(dd['text_preprocessed'])
text=text.replace("stock",' ')
text=text.replace("GME"," ")
text=text.replace("nan",' ')
wordcloud = WordCloud(width=1500, height=500).generate(text)
plt.figure(figsize=(18,8))
plt.imshow(wordcloud, interpolation='bilinear');
plt.title('Happy') ;
plt.axis("off") ;

In [ ]:
# Wordplot for being angry,sad,emotional
fig, ax = plt.subplots(2, 2, figsize=(16, 8))
k=0
j=0
for i in ['Sad','Fear','Surprise','Angry']:
    dd=read_file[read_file['dominant emotion']==i]
    text=" ".join(dd['text_preprocessed'])
    text=text.replace("stock",' ')
    text=text.replace("GME"," ")
    text=text.replace("nan",' ')
    wordcloud = WordCloud(width=1500, height=500).generate(text)

    ax[k,j].imshow(wordcloud, interpolation='bilinear')
    ax[k,j].set_title(i)
    ax[k, j].set_axis_off()
    j+=1
    if j>1:
        k+=1
        j=0